In [ ]:
import requests
import re
import csv
import time
url = "https://jlcpcb.com/api/overseas-pcb-order/v1/shoppingCart/smtGood/selectSmtComponentList/v2"

headers = {
    "Host": "jlcpcb.com",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:130.0) Gecko/20100101 Firefox/130.0",
    "Accept": "application/json, text/plain, */*",
    "Accept-Language": "en-GB,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Content-Type": "application/json",
    "Origin": "https://jlcpcb.com",
    "DNT": "1",
    "Connection": "keep-alive",
    "Referer": "https://jlcpcb.com/parts/basic_parts",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-origin",
    "Priority": "u=0",
    "Pragma": "no-cache",
    "Cache-Control": "no-cache",
}

component_codes: list[int] = []

with open("scraped/ComponentList.csv", "r", newline="") as f:
    reader = csv.reader(f)
    next(reader)  # Skip the header row
    for row in reader:
        component_codes.append(int(row[0]))

for page in range(1,48):
    data = {
        "currentPage": page,
        "pageSize": 100,
        "keyword": None,
        "componentLibraryType": "base",
        "preferredComponentFlag": True,
        "stockFlag": None,
        "stockSort": None,
        "firstSortName": None,
        "secondSortName": None,
        "componentBrand": None,
        "componentSpecification": None,
        "componentAttributes": [],
        "searchSource": "search",
    }

    response = requests.post(url, headers=headers, json=data)

    print(f"Page {page}: {response.status_code} {response.headers}")
    
    unseen_components = 0

    if response.status_code == 200:
        page_components = re.findall(r'"componentCode":"C(\d+)"', response.text)
        for component in page_components:
            component_num = int(component)
            if component_num not in component_codes:
                component_codes.append(component_num)
                unseen_components+=1

    print(f"\tFound {len(page_components)} Components, {unseen_components} Unseen Components")
    
    # time.sleep(0.5)  # Pause (try to not get rate limited)
    
component_codes.sort()
# for number in component_codes:
#     print(number)
    
with open("scraped/ComponentList.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["lcsc"])  # header
    writer.writerows([[code] for code in component_codes])